### KoNLPy
: 한글 텍스트 분석
- :파이썬의 대표적인 한글 형태소 패키지
- :말뭉치를 형태소 어근 단위로 쪼개고 각 형태소에서 품사 태깅(POStagging)을 부착하는 작업 시행

### KoNLPy 실습 -  네이버 영화평점 감성분석

In [1]:
import pandas as pd

train_df = pd.read_csv('ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [2]:
train_df['label'].value_counts()
# 긍정(1), 부정(0)

0    75173
1    74827
Name: label, dtype: int64

In [3]:
import re #정규표현식

train_df = train_df.fillna(' ')

test_df=pd.read_csv('ratings_test.txt',sep='\t')
test_df=test_df.fillna(' ')                        # document 에서 숫자를 공백으로 치환.   
test_df['document']=test_df['document'].apply(lambda x: re.sub(r"\d+"," ",x))                                                 
# re.sub(pattern, repl, string, count=0, flags=0) : string에서 겹치지 않는 pattern의 가장 왼쪽 일치를 repl로 치환
                                                             
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)
train_df.head()

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
# 형태소 분석을 위해 형태소 토큰화 작업 
# sns 한글 형태소 토큰화 작업에 가장 최적화된 Twitter 사용

from konlpy.tag import Twitter
twitter = Twitter()

def tw_tokenizer(text):
    tokens_ko = twitter.morphs(text) # 형태소 분석
    return tokens_ko

C:\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\anaconda3\lib\site-packages\jpype\_core.py:217: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

                            # min_df: 단어가 최소 3번이상 나와야 함. max_df 문서의 90%이상에 나타나는 단어 무시.
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train=tfidf_vect.tfansform(train_df['documnet'])

In [ ]:
lg_cl = LogisticRegression(random_state=0)
params = {'C':[1, 3, 5, 4.5, 5.5, 10]}

# GridSearchCV() : 최적의 파라미터를 찾아주고, 교차검증을 해줌.
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_, 4))

In [ ]:
from sklearn.metrics import accuracy_score

tfidf_matrix_test = tfidf_vect.transform(Test_df['document'])
best_estimator = grid_cv.best_estimator_
preds=best_estimator.predict(tifdf_matrix_test)
print('Logistic Regression 정확도: ', accuracy_score(test_df['label'], preds))